# TCC - Predição de notas capes de programas de pós-graduação
#### Discente: Rodrigo Augusto Valeretto - NUSP: 10684792
#### Orientador: Adenilso da Silva Simão

In [1]:
import pymongo
import pandas as pd

### Definindo variáveis globais que mudam a estrutura do documento

In [2]:
# reset_originais = True    # Define se as bases de dados do banco devem ser reconstruídas com os csvs do zero
reset_originais = False   # Se falso o valor já carregado dos csvs no banco de Originais vai ser utilizado

passa_conceito_bool = True
# passa_conceito_bool = False

# remove_conceito_bool = True
remove_conceito_bool = False

### Definição de funções

In [3]:
def instanciar_collections(sucupira_db):
    progs_c = sucupira_db["Programas"]
    anais_c = sucupira_db["Producoes_anais"]
    artpe_c = sucupira_db["Producoes_artpe"]
    docentes_c = sucupira_db["Docentes"]
    btd_c = sucupira_db["Teses_Dissertacoes"]
    
    return progs_c, anais_c, artpe_c, docentes_c, btd_c

In [4]:
def merge_dfs(*dfs, on):
    last_df = dfs[len(dfs)-1].copy()
    uniques = set()
    for df in dfs:
        uniques.update(set(df[on]))
        
    diff = uniques.difference(set(last_df[on]))
    for i in diff:
        for j in range(len(dfs)-2, -1, -1):
            df = dfs[j]
            last_df = pd.concat([last_df, df[df[on] == i]], ignore_index=False)
    
    return last_df

In [5]:
def instanciar_dataframes_csv(database_string):
    print("Lendo CSVs para", database_string)
    if(database_string == "Sucupira_2017"):
        progs_df_2017 = pd.read_csv("CSVs/br-capes-colsucup-prog-2017-2021-11-10.csv", encoding="cp1252", delimiter=";")
        progs_df_2018 = pd.read_csv("CSVs/br-capes-colsucup-prog-2018-2021-11-10.csv", encoding="cp1252", delimiter=";")
        progs_df_2019 = pd.read_csv("CSVs/br-capes-colsucup-prog-2019-2021-11-10.csv", encoding="cp1252", delimiter=";")
        progs_df_2020 = pd.read_csv("CSVs/br-capes-colsucup-prog-2020-2021-11-10.csv", encoding="cp1252", delimiter=";")
        progs_df = merge_dfs(progs_df_2017, progs_df_2018, progs_df_2019, progs_df_2020, on="CD_PROGRAMA_IES")
        
        doc_2017 = pd.read_csv("CSVs/br-capes-colsucup-docente-2017-2021-11-10.csv", encoding="cp1252", delimiter=";")
        doc_2018 = pd.read_csv("CSVs/br-capes-colsucup-docente-2018-2021-11-10.csv", encoding="cp1252", delimiter=";")
        doc_2019 = pd.read_csv("CSVs/br-capes-colsucup-docente-2019-2021-11-10.csv", encoding="cp1252", delimiter=";")
        doc_2020 = pd.read_csv("CSVs/br-capes-colsucup-docente-2020-2021-11-10.csv", encoding="cp1252", delimiter=";")
        docentes_df = merge_dfs(doc_2017, doc_2018, doc_2019, doc_2020, on="CD_PROGRAMA_IES")
        
        anais_df = pd.read_csv("CSVs/br-colsucup-prod-detalhe-bibliografica-2017a2020-2022-06-30-anais.csv", encoding="cp1252", delimiter=";")
        artpe_df = pd.read_csv("CSVs/br-colsucup-prod-detalhe-bibliografica-2017a2020-2022-06-30-artpe.csv", encoding="cp1252", delimiter=";")
        btd_df_2017 = pd.read_csv("CSVs/br-capes-btd-2017a2020-2021-12-03_2017.csv", encoding="cp1252", delimiter=";", encoding_errors="ignore")
        btd_df_2018 = pd.read_csv("CSVs/br-capes-btd-2017a2020-2021-12-03_2018.csv", encoding="cp1252", delimiter=";", encoding_errors="ignore")
        btd_df_2019 = pd.read_csv("CSVs/br-capes-btd-2017a2020-2021-12-03_2019.csv", encoding="cp1252", delimiter=";", encoding_errors="ignore")
        btd_df_2020 = pd.read_csv("CSVs/br-capes-btd-2017a2020-2021-12-03_2020.csv", encoding="cp1252", delimiter=";", encoding_errors="ignore")
        btd_df = pd.concat([btd_df_2017, btd_df_2018, btd_df_2019, btd_df_2020], ignore_index=True)
    else:
        progs_df_2013 = pd.read_csv("CSVs/br-capes-colsucup-prog-2013a2016-2020-06-12_2013.csv", encoding="cp1252", delimiter=";")
        progs_df_2014 = pd.read_csv("CSVs/br-capes-colsucup-prog-2013a2016-2020-06-12_2014.csv", encoding="cp1252", delimiter=";")
        progs_df_2015 = pd.read_csv("CSVs/br-capes-colsucup-prog-2013a2016-2020-06-12_2015.csv", encoding="cp1252", delimiter=";")
        progs_df_2016 = pd.read_csv("CSVs/br-capes-colsucup-prog-2013a2016-2020-06-12_2016.csv", encoding="cp1252", delimiter=";")
        progs_df = merge_dfs(progs_df_2013, progs_df_2014, progs_df_2015, progs_df_2016, on="CD_PROGRAMA_IES")
        
        doc_2013 = pd.read_csv("CSVs/br-capes-colsucup-docente-2013-2021-03-01.csv", encoding="cp1252", delimiter=";")
        doc_2014 = pd.read_csv("CSVs/br-capes-colsucup-docente-2014-2021-03-01.csv", encoding="cp1252", delimiter=";")
        doc_2015 = pd.read_csv("CSVs/br-capes-colsucup-docente-2015-2021-03-01.csv", encoding="cp1252", delimiter=";")
        doc_2016 = pd.read_csv("CSVs/br-capes-colsucup-docente-2016-2021-03-01.csv", encoding="cp1252", delimiter=";")
        docentes_df = merge_dfs(doc_2013, doc_2014, doc_2015, doc_2016, on="CD_PROGRAMA_IES")
        
        anais_df = pd.read_csv("CSVs/br-capes-colsucup-producao-2013a2016-2017-11-01-bibliografica-anais.csv", encoding="cp1252", delimiter=";")
        artpe_df = pd.read_csv("CSVs/br-colsucup-prod-detalhe-bibliografica-2013a2016-2017-10-01-artpe.csv", encoding="cp1252", delimiter=";")
        btd_df_2013 = pd.read_csv("CSVs/br-capes-btd-2013a2016-2017-12-01_2013.csv", encoding="cp1252", delimiter=";", encoding_errors="ignore")
        btd_df_2014 = pd.read_csv("CSVs/br-capes-btd-2013a2016-2017-12-01_2014.csv", encoding="cp1252", delimiter=";", encoding_errors="ignore")
        btd_df_2015 = pd.read_csv("CSVs/br-capes-btd-2013a2016-2017-12-01_2015.csv", encoding="cp1252", delimiter=";", encoding_errors="ignore")
        btd_df_2016 = pd.read_csv("CSVs/br-capes-btd-2013a2016-2017-12-01_2016.csv", encoding="cp1252", delimiter=";", encoding_errors="ignore")
        btd_df = pd.concat([btd_df_2013, btd_df_2014, btd_df_2015, btd_df_2016], ignore_index=True)
        
    print("CSVs carregados em dataframes")
    return progs_df, anais_df, artpe_df, docentes_df, btd_df

In [6]:
def instanciar_mongo(database_string):
    client = pymongo.MongoClient()
    
    sucupira_db_orig = client[database_string + "_Orig"]
    progs_c_orig, anais_c_orig, artpe_c_orig, docentes_c_orig, btd_c_orig = instanciar_collections(sucupira_db_orig)
    
    sucupira_db = client[database_string]
    progs_c, anais_c, artpe_c, docentes_c, btd_c = instanciar_collections(sucupira_db)
    
    if(reset_originais):
        progs_df, anais_df, artpe_df, docentes_df, btd_df = instanciar_dataframes_csv(database_string)
        client.drop_database(database_string + "_Orig")

        print("Iniciando inserções", database_string)

        progs_r = progs_df.to_dict(orient="records")
        anais_r = anais_df.to_dict(orient="records")
        artpe_r = artpe_df.to_dict(orient="records")
        docentes_r = docentes_df.to_dict(orient="records")
        btd_r = btd_df.to_dict(orient="records")

        progs_c_orig.insert_many(progs_r)
        anais_c_orig.insert_many(anais_r)
        artpe_c_orig.insert_many(artpe_r)
        docentes_c_orig.insert_many(docentes_r)
        btd_c_orig.insert_many(btd_r)

        print("Inserções finalizadas", database_string)
        
    print("Gerando tabelas para alteração", database_string)
    
    progs_c_orig.aggregate([{ "$out": {"db": database_string, "coll": "Programas"} }])
    anais_c_orig.aggregate([{ "$out": {"db": database_string, "coll": "Producoes_anais"} }])
    artpe_c_orig.aggregate([{ "$out": {"db": database_string, "coll": "Producoes_artpe"} }])
    docentes_c_orig.aggregate([{ "$out": {"db": database_string, "coll": "Docentes"} }])    
    btd_c_orig.aggregate([{ "$out": {"db": database_string, "coll": "Teses_Dissertacoes"} }])
    
    print("Finalizado", database_string)
    
    return sucupira_db, sucupira_db_orig, progs_c, anais_c, artpe_c, docentes_c, btd_c

In [7]:
def join_collections(collection, fromString, localField, foreignField, outString="merged_collection"):
    collection.aggregate([
        {
            "$lookup": {
                "from": fromString,
                "localField": localField,
                "foreignField": foreignField,
                "as": "mergedFields"
            }
        },
        {
            "$replaceRoot": { "newRoot": { "$mergeObjects": [ { "$arrayElemAt": [ "$mergedFields", 0 ] }, "$$ROOT" ] } }
        },
        {   "$project": { "mergedFields": 0, "_id": 0 } },
        {   "$out": outString }
    ])

In [8]:
def generate_cols_to_rm_dict(array):
    res = {}
    for value in array:
        res[value] = 1
    return res

In [9]:
def trata_dados(sucupira_db):
    progs_c, anais_c, artpe_c, docentes_c, btd_c = instanciar_collections(sucupira_db)
    
    ## Tratamentos iniciais de dados
    cols_to_rm = ['ID_ADD_FOTO_PROGRAMA_IES', 'ID_ADD_FOTO_PROGRAMA']
    cols_to_rm_dict = generate_cols_to_rm_dict(cols_to_rm)
    
    ## Remove colunas do array acima
    progs_c.update_many(
        {},
        {
            "$unset": cols_to_rm_dict
        }
    )
    
    ## Renomeia campo ano inicio programa para deixa-lo coerente nos dois DFs (2013 e 2017)
    progs_c.update_many(
        {},
        {
            "$rename": { "ANO_INICIO_PROGRAMA": "AN_INICIO_PROGRAMA" }
        }
    )
    
    # Trocando conceitos com letra por valores
    # A = 5
    # B = 4
    # C = 3
    progs_c.update_many(
        {
            "CD_CONCEITO_PROGRAMA": { "$eq": "A" }
        },
        {
            "$set": { "CD_CONCEITO_PROGRAMA": 5 }
        }
    )
    
    progs_c.update_many(
        {
            "CD_CONCEITO_PROGRAMA": { "$eq": "B" }
        },
        {
            "$set": { "CD_CONCEITO_PROGRAMA": 4 }
        }
    )
    
    progs_c.update_many(
        {
            "CD_CONCEITO_PROGRAMA": { "$eq": "C" }
        },
        {
            "$set": { "CD_CONCEITO_PROGRAMA": 3 }
        }
    )
    
    ## Converte os valores da tabela de conceitos para inteiro (Caso estejam como String)
    progs_c.update_many(
        {},
        [
            {
                "$set": {
                    "CD_CONCEITO_PROGRAMA": {
                        "$toInt": "$CD_CONCEITO_PROGRAMA"
                    }
                }
            }
        ]
    )
    

    
    ## Montando infos relevantes para calculo CAPES
    ## Calcula quantidade de produções do tipo anais
    anais_c.aggregate([
        {

            "$group": {
                "_id" : "$CD_PROGRAMA_IES",
                "Qtd prod anais" : {"$sum" : 1}
            }
        },
        {
            "$project": {
                "_id": 0,
                "CD_PROGRAMA_IES": "$_id",
                "QTD_ANAIS": "$Qtd prod anais"
            }
        },
        {
            "$out": "Qtd_prod_anais"
        }
    ])
    
    ## Calcula quantidade de produções do tipo artigos e periódicos
    artpe_c.aggregate([
        {

            "$group": {
                "_id" : "$CD_PROGRAMA_IES",
                "Qtd prod artpe" : {"$sum" : 1}
            }
        },
        {
            "$project": {
                "_id": 0,
                "CD_PROGRAMA_IES": "$_id",
                "QTD_ARTPE": "$Qtd prod artpe"
            }
        },
        {
            "$out": "Qtd_prod_artpe"
        }
    ])
    
    join_collections(progs_c, "Qtd_prod_anais", "CD_PROGRAMA_IES", "CD_PROGRAMA_IES")
    
    aux_c = sucupira_db["merged_collection"]
    
    join_collections(aux_c, "Qtd_prod_artpe", "CD_PROGRAMA_IES", "CD_PROGRAMA_IES")
    
    ## Calcula a duração dos programas de graduação, levando em consideração a data de finalização ou o ano em questão
    # aux_c.aggregate([
    #     {
    #         "$addFields" : {
    #             "DURACAO_PROGRAMA" : {
    #                 "$cond" : {
    #                     "if": { "$eq": ["$DS_SITUACAO_PROGRAMA", "EM DESATIVACAO"] },
    #                     "then": {
    #                         "$subtract" : [  
    #                             { "$toInt": {"$substr" : ["$DT_SITUACAO_PROGRAMA", 5, 4]} },
    #                             { "$toInt": "$AN_INICIO_PROGRAMA" }
    #                         ]
    #                     },
    #                     "else": {
    #                         "$subtract" : [  
    #                             { "$year": "$$NOW" },
    #                             { "$toInt": "$AN_INICIO_PROGRAMA" }
    #                     ]
    #                     }
    #                 }
    #             }
    #         }
    #     },
    #     { "$out" : "merged_collection" }
    # ])
    
    aux_c.aggregate([
        
        {
            "$addFields" : {
                "DURACAO_PROGRAMA" : {
                    "$cond": {
                        "if": 
                        {
                            "$lt":
                            [
                                {
                                    "$subtract" : [
                                        { "$toInt": { "$substr" : ["$DT_SITUACAO_PROGRAMA", 5, 4]} },
                                        { "$toInt": "$AN_INICIO_PROGRAMA" }
                                    ]
                                }, 0]
                        },
                        "then": 0,
                        "else": {
                            "$subtract" :
                            [
                                { "$toInt": { "$substr" : ["$DT_SITUACAO_PROGRAMA", 5, 4]} },
                                { "$toInt": "$AN_INICIO_PROGRAMA" }
                            ]
                        }
                    }
                }
            }
        },
        { "$out" : "merged_collection" }
    ])
    
    ## Calcula a quantidade de docentes por titulação por programa
    docentes_c.aggregate([
        {
            "$group": {
                "_id": "$CD_PROGRAMA_IES",
                "QTD_DOCENTES_DOUTORADO": { "$sum": { "$toInt": {"$eq": [ "$NM_GRAU_TITULACAO", "DOUTORADO" ]} } },
                "QTD_DOCENTES_MESTRADO": { "$sum": { "$toInt": {"$eq": [ "$NM_GRAU_TITULACAO", "MESTRADO" ]} } },
                "QTD_DOCENTES": { "$sum": 1 }
            }
        },
        {
            "$out": "docentes_tit_count"
        }
    ])
    
    btd_c.aggregate([
        {
            "$group": {
                "_id": "$CD_PROGRAMA",
                "QTD_DISCENTES_DOUTORADO": { "$sum": { "$toInt": {"$eq": [ "$ID_GRAU_ACADEMICO", 28 ]} } },
                "QTD_DISCENTES_MESTRADO": { "$sum": { "$toInt": {"$eq": [ "$ID_GRAU_ACADEMICO", 27 ]} } },
                "QTD_DISCENTES": { "$sum": 1 }
            }
        },
        {
            "$out": "discentes_tit_count"
        }
    ])

    join_collections(aux_c, "docentes_tit_count", "CD_PROGRAMA_IES", "_id")
    join_collections(aux_c, "discentes_tit_count", "CD_PROGRAMA_IES", "_id")
    
    aux_c.aggregate([
        {
            "$addFields": {
                "FORMANDOS_POR_DOCENTE": {
                    "$divide": [
                        "$QTD_DISCENTES",
                        "$QTD_DOCENTES"
                    ]
                }
            }
        },
        {
            "$out": "merged_collection"
        }
    ])
    
    ## Preenche a variáveis com zero nos documentos que não tem essas informações
    aux_c.aggregate([
        {
            "$fill": {
                "output": {
                    "QTD_DOCENTES": { "value": 0 },
                    "QTD_DOCENTES_DOUTORADO": { "value": 0 },
                    "QTD_DOCENTES_MESTRADO": { "value": 0 },
                    "QTD_DISCENTES": { "value": 0 },
                    "QTD_DISCENTES_DOUTORADO": { "value": 0 },
                    "QTD_DISCENTES_MESTRADO": { "value": 0 },
                    "FORMANDOS_POR_DOCENTE": { "value": 0 },
                    "QTD_ARTPE": { "value": 0 },
                    "QTD_ANAIS": { "value": 0 }
                }
            }
        },
        { "$out":  "merged_collection" }
    ])
    
    
    ## Exporta todas as alterações anteriores para a tabela de programas
    aux_c.aggregate([
        {
            "$out": "Programas"
        }
    ])

    sucupira_db.drop_collection("Qtd_prod_anais")
    sucupira_db.drop_collection("Qtd_prod_artpe")
    sucupira_db.drop_collection("merged_collection")
    sucupira_db.drop_collection("docentes_tit_count")
    sucupira_db.drop_collection("discentes_tit_count")

In [10]:
def remove_conceitos(sucupira_db):
    progs_c, _, _, _, _ = instanciar_collections(sucupira_db)
    
    progs_c.delete_many({
        "CD_CONCEITO_PROGRAMA": None
    })
    
    
    progs_c.delete_many({
        "CD_CONCEITO_PROGRAMA": {"$lt": 3}
    })

In [11]:
def passa_conceito(sucupira_db_2013, sucupira_db_2017):
    progs_c_2013, anais_c_2013, artpe_c_2013, docentes_c_2013, btd_c_2013 = instanciar_collections(sucupira_db_2013)
    progs_c_2017, anais_c_2017, artpe_c_2017, docentes_c_2017, btd_c_2017 = instanciar_collections(sucupira_db_2017)
    
    progs_c_2017.aggregate([
        {
            "$project": {
                "CD_PROGRAMA_IES": 1,
                "CD_CONCEITO_PROGRAMA_NOVO": "$CD_CONCEITO_PROGRAMA"
            }
        },
        {
            "$out": { "db": "Sucupira_2013", "coll": "merged_collection" }
        }
    ])

    join_collections(progs_c_2013, "merged_collection", "CD_PROGRAMA_IES", "CD_PROGRAMA_IES", outString="Programas")

    progs_c_2013.update_many(
        {},
        [
            {
                "$set": { "CD_CONCEITO_PROGRAMA": "$CD_CONCEITO_PROGRAMA_NOVO" }
            },
            {
                "$unset": "CD_CONCEITO_PROGRAMA_NOVO"
            }
        ]
    )
    
    remove_conceitos(sucupira_db_2013)
    
    remove_conceitos(sucupira_db_2017)
    
    sucupira_db_2013.drop_collection("merged_collection")

### Tratamento dos dados

In [12]:
sucupira_db_2013, sucupira_db_orig_2013, progs_c_2013, anais_c_2013, artpe_c_2013, docentes_c_2013, btd_c_2013 = instanciar_mongo("Sucupira_2013")
sucupira_db_2017, sucupira_db_orig_2017, progs_c_2017, anais_c_2017, artpe_c_2017, docentes_c_2017, btd_c_2017 = instanciar_mongo("Sucupira_2017")

Gerando tabelas para alteração Sucupira_2013
Finalizado Sucupira_2013
Gerando tabelas para alteração Sucupira_2017
Finalizado Sucupira_2017


In [13]:
# for col in anais_df_2013.columns:
#     if("ID" in col or "DH" in col):
#         continue
#     else:
#         print(col)

#### Agregando e tratando dados para database de 2013

In [14]:
trata_dados(sucupira_db_2013)

#### Agregando e tratando dados para database de 2017

In [15]:
trata_dados(sucupira_db_2017)

In [16]:
pd.DataFrame(progs_c_2013.aggregate([
    {
        "$group": {
            "_id": "$CD_CONCEITO_PROGRAMA",
            "count": { "$sum": 1 }
        }
    }
]))

,_id,count
0,7,145
1,1,4
2,3,1819
3,6,270
4,5,615
5,4,1389
6,2,44


In [17]:
pd.DataFrame(progs_c_2017.aggregate([
    {
        "$group": {
            "_id": "$CD_CONCEITO_PROGRAMA",
            "count": { "$sum": 1 }
        }
    }
]))

,_id,count
0,5,1114
1,4,1591
2,2,113
3,6,309
4,7,185
5,1,12
6,3,1580


In [18]:
# A função passa conceito serve para pegar os conceitos da tabela de 2017, que em tese são os
# valores atribuidos pelo capes à de 2013 e salvá-los na tabela de 2013 para que a lógica fique mais simples
# ainda falta remover os documentos que receberem conceito null no merge.
# Ficará comentado para que eu possa validar com o Ades se essa lógica está certa.

if(passa_conceito_bool): 
    passa_conceito(sucupira_db_2013, sucupira_db_2017)
elif(remove_conceito_bool):
    remove_conceitos(sucupira_db_2013)
    remove_conceitos(sucupira_db_2017)

In [19]:
pd.DataFrame(progs_c_2013.aggregate([
    {
        "$group": {
            "_id": "$CD_CONCEITO_PROGRAMA",
            "count": { "$sum": 1 }
        }
    }
]))

,_id,count
0,7,185
1,3,1321
2,6,307
3,4,1508
4,5,773


In [20]:
pd.DataFrame(progs_c_2017.aggregate([
    {
        "$group": {
            "_id": "$CD_CONCEITO_PROGRAMA",
            "count": { "$sum": 1 }
        }
    }
]))

,_id,count
0,4,1591
1,5,1114
2,6,309
3,7,185
4,3,1580
